In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df_houses = pd.read_csv('/kaggle/input/brasilian-houses-to-rent/houses_to_rent_v2.csv')
df_houses.head(5)

In [ ]:
#Renomeando a coluna de valor do aluguel
df_houses.rename(columns={'rent amount (R$)' : 'valor_aluguel'},inplace = True)

# Explorando o valor do Aluguel
## Média dos aluguéis

In [ ]:
df_houses['valor_aluguel'].mean()

# Mediana dos aluguéis

In [ ]:
df_houses['valor_aluguel'].median()

# Desvio Padrão

In [ ]:
df_houses['valor_aluguel'].std()

# Análise Geral

In [ ]:
df_houses['valor_aluguel'].describe()

In [ ]:
plt.figure(figsize=(12,6))
df_houses['valor_aluguel'].plot(kind='hist',bins=100);

In [ ]:
# análise skewness
df_houses['valor_aluguel'].skew()

In [ ]:
# Kurtosis - Curva Leptocurtica (kurtosis > 0)
df_houses['valor_aluguel'].kurtosis()

# Respondendo algumas perguntas sobre os dados

### Perguntas

- Qual a cidade com a média de aluguel mais alta?
- Quantos banheiros existem nas residências com aluguéis maiores?
- Os imóveis mais caros aceitam animal?
- Os imóveis mais caros são mobiliados?

# 1 - Qual a cidade com a média de aluguel mais alta?

In [ ]:
#Qual a cidade com a média de aluguel mais alta?
df_houses.groupby('city')['valor_aluguel'].mean().reset_index().sort_values('valor_aluguel',ascending=False)

### Conforme observado acima, São Paulo é a cidade com o maior valor de aluguel da base utilizada.

# 2 - Quantos banheiros existem em média nas residências com aluguéis maiores?
### Para a análise foi considerado que valores altos de aluguel são acima de R$5.000,00

In [ ]:
#Criando uma nova coluna para classificar alugueis como valores altos e baixos
df_houses['aluguel_alto'] = ['Alto' if x > 5000 else 'Baixo' for x in df_houses['valor_aluguel']]

In [ ]:
#Média do número de banheiros por faixa de aluguel
df_houses.groupby('aluguel_alto')['bathroom'].mean()

In [ ]:
#Análise da quantidade de banheiros por regiões em aluguéis considerados altos
df_houses.groupby(['aluguel_alto','city'])['bathroom'].mean()['Alto']

### As residências com aluguel maior tem em média 3,7 banheiros, contra 1,7 para alugueis menores. Dentre os maiores alugueis, a região do Rio de Janeiro tem a menor média, sendo Belo Horizonte com a maior.

# 3 - Qual a porcentagem dos imóveis com os maiores valores aceitam animais?

In [ ]:
#Filtrando o dataset e fazendo o cálculo da porcentagem
df_houses.query('aluguel_alto == "Alto" & animal == "acept"').shape[0] / df_houses.query('aluguel_alto == "Alto"').shape[0]

In [ ]:
#Avaliando o total de imóveis de alto valor que aceitam animais por cidade
df_houses.query('aluguel_alto == "Alto" & animal == "acept"').groupby('city').count()['animal']

In [ ]:
#Calculo da porcentagem de aceitação por região.
df_houses.query('aluguel_alto == "Alto" & animal == "acept"').groupby('city').count()['animal'] / df_houses.query('aluguel_alto == "Alto"').groupby('city').count()['animal']

### Com base na análise dentre os imóveis de maior valor, 82% aceitam animais, sendo Campinas (88,6%) a região com maior porcentagem de aceitação e Belo Horizonte (81,1%) a de menor.

# 4 - Qual a % dos imóveis com valor maior são mobiliados?

In [ ]:
#Cálculo da %
df_houses.query('aluguel_alto == "Alto" & furniture == "furnished"').shape[0] / df_houses.query('aluguel_alto == "Alto"').shape[0]


In [ ]:
#Calculo da porcentagem por região.
df_houses.query('aluguel_alto == "Alto" & furniture == "furnished"').groupby('city').count()['furniture'] / df_houses.query('aluguel_alto == "Alto"').groupby('city').count()['furniture']

### Em geral apenas 34% dos imóveis com valor acima de R$5000,00 são mobiliados, sendo a região de Belo Horizonte a que apresenta menor taxa (19,9%) e Porto Alegre a maior (48,3%)

# Avaliando a correlação entre as variáveis

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(df_houses.corr(method='spearman'), annot=True);

### Pelo mapa de correlação, observamos que todas as variáveis estão positivamente correlacionadas com o valor do aluguel, sendo que "fire insurance" apresentando uma correlação muito forte e o valor do condomínio (hoa) uma correlação fraca.